In [1]:
import os
import re
import ast
import json
import time

import numpy as np
import pandas as pd

from IPython.display import clear_output
import plotly.express as px

After investigating the ``False`` values, there is only one incomplete error handling process instance. The second was incorrectly set to wrong, because there were two materials that needed to be moved.

# Calculation of Error-Handling Simulation Results

In [2]:
results_dir = "results"
results_path = os.path.join(results_dir, "simulation_results_rules.csv")
results = pd.read_csv(results_path)
agg_results = results.groupby(["idoc_id", "date", "pollex_id", "error_message", "mockups"]).agg(list)
agg_results["messages"] = agg_results["messages"].apply(lambda x: "'"+"\n\n-----\n\n".join(x))
agg_results["process"] = agg_results["process"].apply(lambda x: (sum(x) / len(x)))
agg_results["input_tokens"] = agg_results["input_tokens"].apply(lambda x: (sum(x) / len(x)))
agg_results["output_tokens"] = agg_results["output_tokens"].apply(lambda x: (sum(x) / len(x)))
agg_results["cached_tokens"] = agg_results["cached_tokens"].apply(lambda x: (sum(x) / len(x)))
agg_results["price"] = agg_results["price"].apply(lambda x: (sum(x) / len(x)))
agg_results["completed"] = agg_results["completed"].apply(lambda x: sum(x))
agg_results["failed_attempts"] = agg_results["failed_attempts"].apply(lambda x: (sum(x) / len(x)))
agg_results = agg_results[["process", "input_tokens", "output_tokens", "cached_tokens", "price", "completed", "failed_attempts", "messages", "correct_steps"]].copy()
agg_results["completed_over_all_seeds"] = agg_results["completed"].apply(lambda x: x == 5)
agg_results = agg_results.reset_index()
agg_results["completed_over_all_seeds"].value_counts().sort_index().reset_index()
agg_results["completed_over_all_seeds"] = agg_results["completed_over_all_seeds"].apply(lambda x: "Yes" if x else "No")
agg_results["completed_over_all_seeds"] = agg_results.apply(lambda x: "Yes" if x["completed_over_all_seeds"] == "Yes" else "Yes" if "Material" in x["error_message"] and "and" in x["error_message"] else x["completed_over_all_seeds"], axis=1)

completed = agg_results["completed_over_all_seeds"].value_counts().sort_index().reset_index().rename(columns={"completed_over_all_seeds": "completed"})
completed = completed.groupby("completed").agg("sum") / len(agg_results)
completed["count"] = completed["count"].apply(lambda x: round(x, 2))
display(completed)

,count
completed,
No,0.01
Yes,0.99


In [3]:
agg_results["error_case"] = ""
agg_results.loc[59, "error_case"] = "case2"
def check_for_case(x):
    if x["error_case"] != "":
        return x["error_case"]
    correct_steps = x["correct_steps"] #[ast.literal_eval(y) for y in x["correct_steps"]]
    for step in correct_steps:
        step = ast.literal_eval(step)
        for st in step:
            if st[0] == "send_email" and st[1]["email"] == "error.handler1@company.com" :
                return "case3"
    return "case4"
agg_results["error_case"] = agg_results.apply(lambda x: check_for_case(x), axis=1)
cases = agg_results["error_case"].value_counts().reset_index()
cases

,error_case,count
0,case4,107
1,case3,10
2,case2,1


In [4]:
completed_by_process = agg_results[agg_results["completed_over_all_seeds"] == "Yes"]["process"].value_counts().sort_index().reset_index().rename(columns={"count": "completed"})
all = agg_results["process"].value_counts().sort_index().reset_index().rename(columns={"count": "all"})
completed_by_process = pd.merge(completed_by_process, all, on="process", how="left")
completed_by_process["completed_percentage"] = completed_by_process["completed"] / completed_by_process["all"]
completed_by_process["method"] = "rules"
completed_by_process["completed_percentage"] = completed_by_process["completed_percentage"].apply(lambda x: round(x, 2))
display(completed_by_process)
c = completed_by_process.copy()

,process,completed,all,completed_percentage,method
0,1.0,14,14,1.00,rules
1,2.0,25,25,1.00,rules
2,3.0,8,8,1.00,rules
3,4.0,6,6,1.00,rules
4,5.0,42,42,1.00,rules
5,6.0,7,7,1.00,rules
6,7.0,12,12,1.00,rules
7,8.0,3,4,0.75,rules


In [5]:
PREP = False
if PREP:        
    results_dir = "results"
    results_path = os.path.join(results_dir, "simulation_results_descriptions.csv")
    results = pd.read_csv(results_path)
    def get_incomplete_reason(x):
        clear_output()
        if x["completed"] == True:
            return ""
        print(x["messages"])
        print(x["correct_steps"])
        print(x["incorrect_steps"])
        time.sleep(1)
        x = input("What was the reason for the incomplete result?")
        return x
    results["incomplete_reason"] = results.apply(get_incomplete_reason, axis=1)
    results_dir = "results"
    results_path = os.path.join(results_dir, "simulation_results_descriptions_reasons.csv")
    results = pd.read_csv(results_path)
    results.incomplete_reason.value_counts().reset_index()
    results.incomplete_reason.value_counts().reset_index()

In [6]:
results_dir = "results"
results_path = os.path.join(results_dir, "simulation_results_descriptions_prep.csv")
results = pd.read_csv(results_path)
results["incomplete_reason"] = results["incomplete_reason"].fillna("")
agg_results = results.groupby(["idoc_id", "date", "pollex_id", "error_message", "mockups"]).agg(list)
agg_results["messages"] = agg_results["messages"].apply(lambda x: "'"+"\n\n-----\n\n".join(x))
agg_results["process"] = agg_results["process"].apply(lambda x: (sum(x) / len(x)))
agg_results["input_tokens"] = agg_results["input_tokens"].apply(lambda x: (sum(x) / len(x)))
agg_results["output_tokens"] = agg_results["output_tokens"].apply(lambda x: (sum(x) / len(x)))
agg_results["cached_tokens"] = agg_results["cached_tokens"].apply(lambda x: (sum(x) / len(x)))
agg_results["price"] = agg_results["price"].apply(lambda x: (sum(x) / len(x)))
agg_results["completed"] = agg_results["completed"].apply(lambda x: sum(x))
agg_results["failed_attempts"] = agg_results["failed_attempts"].apply(lambda x: (sum(x) / len(x)))
agg_results["incomplete_reason"] = agg_results["incomplete_reason"].apply(lambda x: " ".join([str(y) for y in x]).strip())
agg_results = agg_results[["process", "input_tokens", "output_tokens", "cached_tokens", "price", "completed", "failed_attempts", "messages", "incomplete_reason", "correct_steps"]].copy()
agg_results["completed_over_all_seeds"] = agg_results["completed"].apply(lambda x: x == 5)
agg_results = agg_results.reset_index()
agg_results["completed_over_all_seeds"] = agg_results["completed_over_all_seeds"].apply(lambda x: "Yes" if x else "No")
agg_results["completed_over_all_seeds"] = agg_results.apply(lambda x: "Yes" if x["incomplete_reason"] == "" else "No", axis=1)

completed = agg_results["completed_over_all_seeds"].value_counts().sort_index().reset_index().rename(columns={"completed_over_all_seeds": "completed"})
completed = completed.groupby("completed").agg("sum") / len(agg_results)
completed["count"] = completed["count"].apply(lambda x: round(x, 2))
display(completed)

,count
completed,
No,0.16
Yes,0.84


In [7]:
completed_by_process = agg_results[agg_results["completed_over_all_seeds"] == "Yes"]["process"].value_counts().sort_index().reset_index().rename(columns={"count": "completed"})
all = agg_results["process"].value_counts().sort_index().reset_index().rename(columns={"count": "all"})
completed_by_process = pd.merge(completed_by_process, all, on="process", how="left")
completed_by_process["completed_percentage"] = completed_by_process["completed"] / completed_by_process["all"]
completed_by_process["method"] = "descriptions"
completed_by_process["completed_percentage"] = completed_by_process["completed_percentage"].apply(lambda x: round(x, 2))
display(completed_by_process)
c1 = completed_by_process.copy()

,process,completed,all,completed_percentage,method
0,1.0,5,14,0.36,descriptions
1,2.0,22,25,0.88,descriptions
2,3.0,7,8,0.88,descriptions
3,4.0,3,6,0.50,descriptions
4,5.0,42,42,1.00,descriptions
5,6.0,7,7,1.00,descriptions
6,7.0,12,12,1.00,descriptions
7,8.0,1,4,0.25,descriptions


In [8]:
c = pd.concat([c, c1]).sort_values(["process", "method"])
c["completed_percentage"] = c["completed_percentage"].apply(lambda x: round(x, 2) * 100)

In [9]:
fig = px.bar(
    c, 
    x='process', 
    y='completed_percentage', 
    color='method', 
    barmode='group', 
    #title="Percentage of completed processes",
    labels={
        "process": "Process ID",
        "completed_percentage": "Percentage of completed processes",
        "method": "LLM agent information type",
    },
    color_discrete_sequence=px.colors.sequential.Blues[3:5],
    text=[f'{int(j)}% ({i}/{z})' for i,j,z in zip(c["completed"], c["completed_percentage"], c["all"])]
)
fig.update_layout(legend=dict(
    orientation="h",
    #yanchor="bottom",
    #y=-0.25,
    #xanchor="right",
    #x=0.47
))
fig.update_traces(textposition='inside')
config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'simulation_results_process_automation',
    'height': 600,
    'width': 1000,
    'scale':10 # Multiply title/legend/axis/canvas sizes by this factor
  }
}
fig.show(config=config)

In [10]:
results_dir = "results"
results_path = os.path.join(results_dir, "simulation_results_descriptions_prep.csv")
results = pd.read_csv(results_path)
results["incomplete_reason"] = results["incomplete_reason"].fillna("")
agg_results = results.groupby(["idoc_id", "date", "pollex_id", "error_message", "mockups"]).agg(list)
agg_results["messages"] = agg_results["messages"].apply(lambda x: "'"+"\n\n-----\n\n".join(x))
agg_results["process"] = agg_results["process"].apply(lambda x: (sum(x) / len(x)))
agg_results["input_tokens"] = agg_results["input_tokens"].apply(lambda x: (sum(x) / len(x)))
agg_results["output_tokens"] = agg_results["output_tokens"].apply(lambda x: (sum(x) / len(x)))
agg_results["cached_tokens"] = agg_results["cached_tokens"].apply(lambda x: (sum(x) / len(x)))
agg_results["price"] = agg_results["price"].apply(lambda x: (sum(x) / len(x)))
agg_results["completed"] = agg_results["completed"].apply(lambda x: sum(x))
agg_results["failed_attempts"] = agg_results["failed_attempts"].apply(lambda x: (sum(x) / len(x)))
agg_results["incomplete_reason"] = agg_results["incomplete_reason"].apply(lambda x: " ".join([str(y) for y in x]).strip())
agg_results = agg_results[["process", "input_tokens", "output_tokens", "cached_tokens", "price", "completed", "failed_attempts", "messages", "incomplete_reason", "correct_steps"]].copy()
agg_results["completed_over_all_seeds"] = agg_results["completed"].apply(lambda x: x == 5)
agg_results = agg_results.reset_index()
agg_results["completed_over_all_seeds"] = agg_results["completed_over_all_seeds"].apply(lambda x: "Yes" if x else "No")
agg_results["completed_over_all_seeds"] = agg_results.apply(lambda x: "Yes" if x["incomplete_reason"] == "" else "No", axis=1)

completed = agg_results["completed_over_all_seeds"].value_counts().sort_index().reset_index().rename(columns={"completed_over_all_seeds": "completed"})
display(completed)
completed = completed.groupby("completed").agg("sum") / len(agg_results)
completed["count"] = completed["count"].apply(lambda x: round(x, 2))
display(completed)

,completed,count
0,No,19
1,Yes,99


,count
completed,
No,0.16
Yes,0.84


In [11]:
PREP = False
if PREP:
    def get_incomplete_reason(x):
        clear_output()
        if x["completed_over_all_seeds"] == "Yes":
            return ""
        print(x["messages"])
        time.sleep(1)
        x = input("What was the reason for the incomplete result?")
        return x
    agg_results["error_case"] = agg_results.apply(get_incomplete_reason, axis=1)
    agg_results.to_csv("results/simulation_results_descriptions_prep_cases.csv", index=False)

In [12]:
agg_results = pd.read_csv("results/simulation_results_descriptions_prep_cases.csv")
agg_results["error_case"] = agg_results["error_case"].fillna("")
def check_for_case(x):
    if x["error_case"] != "":
        return x["error_case"]
    correct_steps = ast.literal_eval(x["correct_steps"]) #[ast.literal_eval(y) for y in x["correct_steps"]]
    for step in correct_steps:
        step = ast.literal_eval(step)
        for st in step:
            if st[0] == "send_email" and st[1]["email"] == "error.handler1@company.com" :
                return "case3"
    return "case4"
agg_results["error_case_tmp"] = agg_results.apply(lambda x: check_for_case(x), axis=1)
display(agg_results[agg_results["error_case_tmp"] == "case1"])
agg_results["error_case_tmp"].value_counts().reset_index()

,idoc_id,date,pollex_id,error_message,mockups,process,input_tokens,output_tokens,cached_tokens,price,completed,failed_attempts,messages,correct_steps,incomplete_reason,completed_over_all_seeds,error_case,error_case_tmp
4,19467893,2023-01-10 00:00:00,98127.0,G/L account 404302 is not provided for in comp...,"{""RULE_BASE"": """", ""RPA_FIELD_VALUE"": {}, ""RFC_...",4.0,10956.0,738.4,9216.0,2.32540,1,0.6,'================================ Human Messag...,"['[[""retrieve_description_to_solve"", {""error_m...","['', 'description misinterpretation', 'descrip...",No,case1,case1
16,46653730,2023-02-27 00:00:00,38466.0,G/L account 506301 is not provided for in comp...,"{""RULE_BASE"": """", ""RPA_FIELD_VALUE"": {}, ""RFC_...",4.0,11384.0,660.8,9984.0,2.25880,4,0.2,'================================ Human Messag...,"['[[""retrieve_description_to_solve"", {""error_m...","['', '', '', '', 'description misinterpretation']",No,case1,case1
105,67518907,2023-04-21 00:00:00,74741.0,G/L account 505401 is not provided for in comp...,"{""RULE_BASE"": """", ""RPA_FIELD_VALUE"": {}, ""RFC_...",4.0,13073.0,731.4,12032.0,2.49565,3,1.0,'================================ Human Messag...,"['[[""retrieve_description_to_solve"", {""error_m...","['description misinterpretation', '', '', 'des...",No,case1,case1


,error_case_tmp,count
0,case4,90
1,case2,16
2,case3,9
3,case1,3


In [13]:
results_dir = "results"
results_path = os.path.join(results_dir, "simulation_results_rules.csv")
results_0 = pd.read_csv(results_path)
results_path = os.path.join(results_dir, "simulation_results_descriptions_prep.csv")
results_1 = pd.read_csv(results_path)
results_path = os.path.join(results_dir, "simulation_results_descriptions_user_requests.csv")
results_2 = pd.read_csv(results_path)
results_path = os.path.join(results_dir, "simulation_results_rules_user_requests.csv")
results_3 = pd.read_csv(results_path)
results = pd.concat([results_0, results_1, results_2, results_3,])
results.price.mean().round(2)

1.72

# Results of the Synthetic User Requests Simulation

In [14]:
results_dir = "results"
results_path = os.path.join(results_dir, "simulation_results_rules_user_requests.csv")
results0 = pd.read_csv(results_path)
print(len(results0))
results0["scale"] = results0.tasks.apply(lambda x: "1-5" if x < 6 else "6-10" if x < 11 else "11-15")
results0["scale_id"] = results0.scale.apply(lambda x: 0 if x == "1-5" else 1 if x == "6-10" else 2)
results0["method"] = "rules"
c0 = round(results0.groupby(["scale", "scale_id", "method"]).completed.value_counts() / results0.scale.value_counts(), 2) * 100
display(round(results0.completed.value_counts() / len(results0), 2))

54


completed
1    0.85
0    0.15
Name: count, dtype: float64

In [ ]:
CHECK_ERRORS = False
if CHECK_ERRORS:  
    for i, row in results0[results0["completed"] == False].iterrows():
        print(row["id"])
        print(row["results"])
        print("-----")
        print("-----")
        print()
        x = input("Press Enter to continue. Type x to leave.")
        if x == "x":
            break
        clear_output()

In [23]:
results_dir = "results"
results_path = os.path.join(results_dir, "simulation_results_descriptions_user_requests.csv")
results1 = pd.read_csv(results_path)
print(len(results1))
results1["scale"] = results1.tasks.apply(lambda x: "1-5" if x < 6 else "6-10" if x < 11 else "11-15")
results1["scale_id"] = results1.scale.apply(lambda x: 0 if x == "1-5" else 1 if x == "6-10" else 2)
results1["method"] = "descriptions"
c1 = round(results1.groupby(["scale", "scale_id", "method"]).completed.value_counts() / results1.scale.value_counts(), 2) * 100
display(round(results1.completed.value_counts() / len(results1), 2))

54


completed
1    0.76
0    0.24
Name: count, dtype: float64

In [ ]:
CHECK_ERRORS = False
if CHECK_ERRORS:   
    for i, row in results1[results1["completed"] == False].iterrows():
        print(row["id"])
        print(row["results"])
        print("-----")
        print("-----")
        print()
        x = input("Press Enter to continue. Type x to leave.")
        if x == "x":
            break
        clear_output()

In [16]:
c = pd.concat([c0, c1])
c = c.reset_index()
c = c[c["completed"] == 1]
c = c.sort_values(["scale_id", "method"])
display(c)
fig = px.bar(
    c, 
    x='scale', 
    y='count', 
    color='method', 
    barmode='group', 
    #title="Percentage of completed processes",
    labels={
        "scale": "Process Scale (Tasks)",
        "count": "Percentage of completed user requests",
        "method": "LLM agent information type",
    },
    color_discrete_sequence=px.colors.sequential.Greens[2:4],
    text=[f'{int(j)}%' for j in c["count"]]
)
fig.update_layout(legend=dict(
    orientation="h",
    #yanchor="bottom",
    #y=-0.25,
    #xanchor="right",
    #x=0.8
))
fig.update_traces(textposition='inside')
config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'simulation_results_user_requests',
    'height': 600,
    'width': 1000,
    'scale':10 # Multiply title/legend/axis/canvas sizes by this factor
  }
}
fig.show(config=config)

,scale,scale_id,method,completed,count
6,1-5,0,descriptions,1,81.0
0,1-5,0,rules,1,88.0
10,6-10,1,descriptions,1,75.0
4,6-10,1,rules,1,85.0
8,11-15,2,descriptions,1,72.0
2,11-15,2,rules,1,83.0
